In [1]:
import numpy as np
import pandas as pd
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CEBinaryClassificationEvaluator
from sentence_transformers import InputExample
import torch
import math

In [2]:
model_name = 'microsoft/deberta-v3-large' # base model, use 'vectara/hallucination_evaluation_model' if you want to further fine-tune ours

model = CrossEncoder(model_name, num_labels=1, max_length = 512, automodel_args={'ignore_mismatched_sizes':True})


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /opt/conda did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['pooler.dense.bias', 'classifier.bias', 'pooler.dense.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a s

In [3]:
df_train = pd.read_csv("")

In [4]:
df_train

,prompt,completion,output
0,What is the airspeed velocity of an unladen sw...,Insect flight its wing frequency (f) about 40 ...,[Irrelevant]
1,There is a sound synthesis technique called Fr...,Phase distortion synthesis frequency of the re...,[Relevant]
2,What is the tallest building in the world?,Council on Tall Buildings and Urban Habitat th...,[Irrelevant]
3,Why is light affected by gravity even if it do...,Gravity Probe A does not seem to travel on a h...,[Relevant]
4,Which programming languages support variable v...,Variable shadowing of the language. One of the...,[Relevant]
...,...,...,...
19312,On a scale 1-10 how do humans rate the pain re...,Dynorphin was measured by tail-flick latency. ...,[Irrelevant]
19313,What is the Splunk SPL to determine statistics...,Splunk to connect directly to applications and...,[Irrelevant]
19314,tell me a story about a boy named james,"James Ramsey Ullman ""High Conquest"" was the fi...",[Irrelevant]
19315,what have you done so far in you life,"Harmful influence of technology essay ""No Need...",[Irrelevant]


In [5]:
df_train["output"] = [1 if i == '[Relevant]' else 0 for i in df_train["output"]]

In [6]:
train_examples = []
for i, row in df_train.iterrows():
    train_examples.append(InputExample(texts=[row['completion'], row['prompt']], label=int(row['output'])))

In [7]:
test_evaluator = CEBinaryClassificationEvaluator.from_input_examples(train_examples, name='test_eval')

In [8]:
num_epochs = 2
model_save_path = "./model_dump"
train_dataloader = torch.utils.data.DataLoader(train_examples, shuffle=True, batch_size=2)
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
model.fit(train_dataloader=train_dataloader,
          evaluator=test_evaluator,
          epochs=num_epochs,
          evaluation_steps=10_000,
          warmup_steps=warmup_steps,
          output_path=model_save_path,
          show_progress_bar=True)


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/9659 [00:00<?, ?it/s]

Iteration:   0%|          | 0/9659 [00:00<?, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
model.save("topic_check_models/deberta_cross_encoder_08_12_2023")